In [1]:
import numpy as np
import pandas as pd


In [2]:
# Входные данные
input_data = np.array(
    [
        [5, 4, 5, 3, 5, 0, 5, 5, 4, 3, 5],
        [5, 5, 5, 3, 5, 0, 5, 4, 4, 2, 5],
        [5, 4, 4, 2, 5, 0, 5, 3, 3, 0, 3],
        [5, 3, 3, 0, 3, 0, 5, 4, 3, 3, 5],
        [5, 0, 4, 0, 0, 0, 5, 4, 4, 1, 5],
        [4, 5, 5, 3, 1, 0, 5, 0, 4, 1, 0],
    ],
    dtype=float,
)
input_data

array([[5., 4., 5., 3., 5., 0., 5., 5., 4., 3., 5.],
       [5., 5., 5., 3., 5., 0., 5., 4., 4., 2., 5.],
       [5., 4., 4., 2., 5., 0., 5., 3., 3., 0., 3.],
       [5., 3., 3., 0., 3., 0., 5., 4., 3., 3., 5.],
       [5., 0., 4., 0., 0., 0., 5., 4., 4., 1., 5.],
       [4., 5., 5., 3., 1., 0., 5., 0., 4., 1., 0.]])

# Формирование рекомендаций для пользователя, уже оценившего другие товары


In [3]:
user_index = 3
users_indices = [i for i in range(input_data.shape[1])]

In [4]:
# Не оцененные пользователем продукты.
not_rated_products_indices = tuple(np.where(input_data[:, user_index] == 0)[0])
not_rated_products_indices

(np.int64(3), np.int64(4))

In [5]:
# Поиск пользователей, которые также не оценили те же продукты.
users_without_rating_on_product = np.argwhere(
    np.all(input_data[not_rated_products_indices, :], axis=0) == 0
)
users_without_rating_on_product

array([[1],
       [3],
       [4],
       [5]])

In [6]:
# Исключаем пользователя из выборки тех, кто не оценил продукты.
exclude_users = (
    users_without_rating_on_product[~np.all(users_without_rating_on_product == user_index, axis=1), :]
)
exclude_users

array([[1],
       [4],
       [5]])

In [7]:
# Получение матрицы из оценок тех пользователей, кто оценил нужные товары.
users_rankings_for_similarity = np.delete(input_data, exclude_users, axis=1)
# Исключение индексов удаленных пользователей.
users_indices_of_rankings_for_similarity = [i for i in users_indices if i not in exclude_users[:, 0]]
# Получение матрицы оценок по тем продуктам, которые были оценены пользователем.
pd.DataFrame(users_rankings_for_similarity, columns=users_indices_of_rankings_for_similarity)

,0,2,3,6,7,8,9,10
0,5.0,5.0,3.0,5.0,5.0,4.0,3.0,5.0
1,5.0,5.0,3.0,5.0,4.0,4.0,2.0,5.0
2,5.0,4.0,2.0,5.0,3.0,3.0,0.0,3.0
3,5.0,3.0,0.0,5.0,4.0,3.0,3.0,5.0
4,5.0,4.0,0.0,5.0,4.0,4.0,1.0,5.0
5,4.0,5.0,3.0,5.0,0.0,4.0,1.0,0.0


In [8]:
def get_cosine_similarity(array_1: np.ndarray[int, int], array_2: np.ndarray[int, int]) -> float:
    """Функция подсчета косинусного подобия."""
    return np.dot(array_1, array_2) / (np.linalg.norm(array_1) * np.linalg.norm(array_2))


def get_cosine_similarity_matrix() -> np.ndarray[[int, int], float]:
    """Получение матрицы схожести."""
    users_count = users_rankings_for_similarity.shape[1]
    users_similarity_matrix: np.ndarray[(int, int), float] = np.full(
        (users_count, users_count),
        0,
        dtype=float,
    )

    users_input_data = users_rankings_for_similarity.T

    for i in range(users_count):
        for j in range(users_count):
            if i != j:
                similarity_value = get_cosine_similarity(users_input_data[i], users_input_data[j])
                users_similarity_matrix[i, j] = similarity_value

    return users_similarity_matrix


In [9]:
# Получение матрицы косинусного подобия пользователей.
similarity_matrix = get_cosine_similarity_matrix()
pd.DataFrame(similarity_matrix, columns=users_indices_of_rankings_for_similarity)

,0,2,3,6,7,8,9,10
0,0.000000,0.977398,0.786526,0.997041,0.930001,0.985801,0.842327,0.927631
1,0.977398,0.000000,0.883824,0.985527,0.871531,0.994571,0.814955,0.862640
2,0.786526,0.883824,0.000000,0.806559,0.654525,0.833032,0.659912,0.619310
3,0.997041,0.985527,0.806559,0.000000,0.901670,0.991837,0.833333,0.899371
4,0.930001,0.871531,0.654525,0.901670,0.000000,0.890244,0.879128,0.994278
5,0.985801,0.994571,0.833032,0.991837,0.890244,0.000000,0.834044,0.888503
6,0.842327,0.814955,0.659912,0.833333,0.879128,0.834044,0.000000,0.879820
7,0.927631,0.862640,0.619310,0.899371,0.994278,0.888503,0.879820,0.000000


In [10]:
# Поиск непохожих пользователей по пороговому значению.
min_similarity_value = 0.8
user_index_in_similarity_matrix = users_indices_of_rankings_for_similarity.index(user_index)
not_similar_users = np.argwhere(similarity_matrix[user_index_in_similarity_matrix, :] < min_similarity_value)
not_similar_users = not_similar_users[~np.all(not_similar_users == user_index_in_similarity_matrix, axis=1), :]
not_similar_users

array([[0],
       [4],
       [6],
       [7]])

In [11]:
# Удаление из матрицы косинусного подобия непохожих пользователей.
similar_users_similarity = np.delete(similarity_matrix, not_similar_users, axis=1)
similar_users_similarity = np.delete(similar_users_similarity, not_similar_users, axis=0)
users_indices_of_similarity_for_similar_users = [
    elem for i, elem in enumerate(users_indices_of_rankings_for_similarity)
    if i not in not_similar_users
]
pd.DataFrame(
    similar_users_similarity,
    columns=users_indices_of_similarity_for_similar_users,
    index=users_indices_of_similarity_for_similar_users,
)

,2,3,6,8
2,0.000000,0.883824,0.985527,0.994571
3,0.883824,0.000000,0.806559,0.833032
6,0.985527,0.806559,0.000000,0.991837
8,0.994571,0.833032,0.991837,0.000000


In [12]:
# Матрица предпочтений пользователей, которые оценили товары, не оцененные пользователем.
users_rankings = np.delete(input_data, users_without_rating_on_product, axis=1)
users_indices_of_user_rankings = [i for i in users_indices if i not in users_without_rating_on_product]
pd.DataFrame(users_rankings, columns=users_indices_of_user_rankings)

,0,2,6,7,8,9,10
0,5.0,5.0,5.0,5.0,4.0,3.0,5.0
1,5.0,5.0,5.0,4.0,4.0,2.0,5.0
2,5.0,4.0,5.0,3.0,3.0,0.0,3.0
3,5.0,3.0,5.0,4.0,3.0,3.0,5.0
4,5.0,4.0,5.0,4.0,4.0,1.0,5.0
5,4.0,5.0,5.0,0.0,4.0,1.0,0.0


In [13]:
# Матрица предпочтений похожих пользователей, которые оценили товар.
not_similar_users_from_rankings = [
    i for i, elem in enumerate(users_indices_of_user_rankings)
    if elem not in users_indices_of_similarity_for_similar_users
]
users_indices_of_similarity_for_similar_users = [i for i in users_indices_of_similarity_for_similar_users if
                                                 i != user_index]

similar_users_rankings = np.delete(users_rankings, not_similar_users_from_rankings, axis=1)
pd.DataFrame(similar_users_rankings, columns=users_indices_of_similarity_for_similar_users)

,2,6,8
0,5.0,5.0,4.0
1,5.0,5.0,4.0
2,4.0,5.0,3.0
3,3.0,5.0,3.0
4,4.0,5.0,4.0
5,5.0,5.0,4.0


In [14]:
# Расчет среднего арифметического пользовательского рейтинга.
avg_user_rating = input_data[np.nonzero(input_data[:, user_index]), user_index].mean()
print(f'Средний пользовательский рейтинг пользователя #{user_index}: {avg_user_rating}')

Средний пользовательский рейтинг пользователя #3: 2.75


In [15]:
# Рекомендуем пользователю неоцененные товары, если расчетный рейтинг >= среднему рейтингу товаров пользователя.
for not_rated_product_index in not_rated_products_indices:
    print(f'Продукт {not_rated_product_index}:')
    avg_rating = users_rankings[not_rated_product_index, :].mean()
    print(f'Средняя оценка продукта от пользователей: {avg_rating}')
    cos_similarity = similarity_matrix[np.nonzero(
        similarity_matrix[:, user_index_in_similarity_matrix]
    ), user_index_in_similarity_matrix]
    rating_diff = users_rankings[not_rated_product_index, :] - avg_rating
    print(f'Дифференциация рейтингов по продукту: {rating_diff}')
    calculated_rating = avg_user_rating + np.sum(rating_diff * cos_similarity) / np.sum(np.abs(cos_similarity))
    print(f'Рассчитанная оценка: {calculated_rating}')
    print(f'Вердикт: {"РЕКОМЕНДОВАН" if calculated_rating > avg_user_rating else "НЕ РЕКОМЕНДОВАН"} '
          f'пользователю #{user_index}.\n')


Продукт 3:
Средняя оценка продукта от пользователей: 4.0
Дифференциация рейтингов по продукту: [ 1. -1.  1.  0. -1. -1.  1.]
Рассчитанная оценка: 2.718653198066379
Вердикт: НЕ РЕКОМЕНДОВАН пользователю #3.

Продукт 4:
Средняя оценка продукта от пользователей: 4.0
Дифференциация рейтингов по продукту: [ 1.  0.  1.  0.  0. -3.  1.]
Рассчитанная оценка: 2.7943693920298824
Вердикт: РЕКОМЕНДОВАН пользователю #3.



# Формирование рекомендаций для нового пользователя

In [16]:
user_index = 5

In [17]:
# Заменяем 0 в матрице на NaN для того, чтобы значения не учитывались в расчете среднего значения рейтинга по продукту.
nan_instead_zeros_input_data = input_data.copy()
nan_instead_zeros_input_data[nan_instead_zeros_input_data == 0] = np.nan
nan_instead_zeros_input_data

array([[ 5.,  4.,  5.,  3.,  5., nan,  5.,  5.,  4.,  3.,  5.],
       [ 5.,  5.,  5.,  3.,  5., nan,  5.,  4.,  4.,  2.,  5.],
       [ 5.,  4.,  4.,  2.,  5., nan,  5.,  3.,  3., nan,  3.],
       [ 5.,  3.,  3., nan,  3., nan,  5.,  4.,  3.,  3.,  5.],
       [ 5., nan,  4., nan, nan, nan,  5.,  4.,  4.,  1.,  5.],
       [ 4.,  5.,  5.,  3.,  1., nan,  5., nan,  4.,  1., nan]])

In [18]:
# Получение массива средних значений рейтинга по продуктам.
averages_products_rating = np.nanmean(nan_instead_zeros_input_data, axis=1)
averages_products_rating

array([4.4       , 4.3       , 3.77777778, 3.77777778, 4.        ,
       3.5       ])

In [19]:
max_rated_product_index = np.argmax(averages_products_rating)
print(f'Рекомендовать новому пользователю продукт #{max_rated_product_index} '
      f'(средний рейтинг == {averages_products_rating[max_rated_product_index]}).')

Рекомендовать новому пользователю продукт #0 (средний рейтинг == 4.4).
